# Combine CSVs For Processing
Combines the CSVs so all relevant data is on one line and the data is placed based on linking keys through the different CSVs

Everything from this markdown cell to the next one is just a sanity check to make sure the chunking for the CSVs is not giving an odd output. Don't run unless you have some reason to check.

# Combine with Dask

We use dask because it automatically chunks data

We need to determine what features we want to be able to determine which CSVs we need to merge

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType

In [2]:
# change to be reflective of your environment
data_dir = '/home/cole/Workspace/School/Capstone/data/first_data_set/TestData/'

In [3]:
# initiates a SparkContext which is necessary for accessing data in Spark
sc = SparkContext()
sqlContext = SQLContext(sc)
# change to match your environment
output_dir = data_dir + "/merge_data"

In [4]:
SDRO = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SD_RECON_ORDER.csv'])
SS_POOLS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SS_POOLS.csv'])
AFBF = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/AF_BITFILES.csv"])
BACKUP_OBJECTS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/BACKUP_OBJECTS.csv"])

In [ ]:
full_outer_join = BACKUP_OBJECTS.join(AFBF, BACKUP_OBJECTS.OBJID == AFBF.BFID,how='left') # Could also use 'full_outer'
full_outer_join = full_outer_join.join(SDRO, ['OBJID'],how='left') # Could also use 'full_outer'

In [ ]:
SDRO = None
SS_POOLS = None
AFBF = None
BACKUP_OBJECTS = None

In [ ]:
df = full_outer_join
df = df.filter(df.POOLID. isNotNull())
df = df.withColumn("POOLID", df["POOLID"].cast("int"))
df = df.filter(df.ATTRLENGTH. isNotNull())
df = df.withColumn("ATTRLENGTH", df["ATTRLENGTH"].cast("int"))
df = df.filter(df.BFSIZE. isNotNull())
df = df.withColumn("BFSIZE", df["BFSIZE"].cast("int"))
df = df.filter(df.HDRSIZE. isNotNull())
df = df.withColumn("HDRSIZE", df["HDRSIZE"].cast("int"))
df = df.filter(df.OBJID. isNotNull())
df = df.withColumn("OBJID", df["OBJID"].cast("int"))

In [ ]:
df.select("OBJID", "POOLID", "ATTRLENGTH", "BFSIZE", "HDRSIZE").write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/merge_data/4_features")

In [ ]:
sc.stop()